## Nettoyage des données des Valeurs Foncières 2023 

Notre fichier étant un fichier texte, nous avons besoin de délimiter les données entres elles. Nous pouvons apercevoir que les données sont séparées par | , ce sera notre delimiter. 

In [3]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

df_2023 = pd.read_csv('ValeursFoncieres-2023.txt', delimiter='|', low_memory=False)
threshold = 0.5

df_2023 = df_2023[df_2023.columns[df_2023.isnull().mean() < threshold]]
df_2023 = df_2023.loc[df_2023.isnull().mean(axis=1) < threshold]

print("DataFrame après nettoyage : ", df_2023.shape)
df_2023.info()
df_2023.head()


DataFrame après nettoyage :  (3773998, 21)
<class 'pandas.core.frame.DataFrame'>
Index: 3773998 entries, 0 to 3799406
Data columns (total 21 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   No disposition             int64  
 1   Date mutation              object 
 2   Nature mutation            object 
 3   Valeur fonciere            object 
 4   No voie                    float64
 5   Type de voie               object 
 6   Code voie                  object 
 7   Voie                       object 
 8   Code postal                float64
 9   Commune                    object 
 10  Code departement           object 
 11  Code commune               int64  
 12  Section                    object 
 13  No plan                    int64  
 14  Nombre de lots             int64  
 15  Code type local            float64
 16  Type local                 object 
 17  Surface reelle bati        float64
 18  Nombre pieces principales  float64
 19  Natu

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,1,05/01/2023,Vente,"1070000,00",184.0,ALL,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,...,354,BD,334,1,3.0,Dépendance,0.0,0.0,NaN,NaN
1,1,05/01/2023,Vente,"1070000,00",159.0,ALL,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,...,354,BD,336,1,2.0,Appartement,233.0,8.0,NaN,NaN
2,1,05/01/2023,Vente,"1070000,00",159.0,ALL,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,...,354,BD,336,1,3.0,Dépendance,0.0,0.0,NaN,NaN
3,1,03/01/2023,Vente,"152200,00",2914.0,RTE,0107,DE PONCIN,1450.0,SERRIERES-SUR-AIN,...,404,D,2317,0,1.0,Maison,64.0,3.0,S,988.0
4,1,05/01/2023,Vente,"269000,00",427.0,CHE,0040,DE L'AUBEPIN,1800.0,SAINT-JEAN-DE-NIOST,...,361,B,2405,0,1.0,Maison,73.0,3.0,S,835.0


Nous gardons les variables suivantes : 

- Date mutation : pour les analyses temporelles des prix dépendant des saisons ou du temps, elle indique la date de la transaction immobilière.

- Nature mutation :

- Valeur foncière : 

- Code postal/Commune/Code commune :

- Nombre de lots :

- Code type local/Type local : 

- Surface réelle bâtie/Nombre prièces principales/Surface terrain : 

In [4]:
columns_to_delete=['No disposition', 'No voie', 'Type de voie', 'Code voie', 'Code plan', 'Section','No plan', 'Nature culture']

df_2023=df_2023.drop(columns_to_delete, errors='ignore', axis=1)
# Supprimer les lignes où il n'y a pas de 'valeur_foncière'
df_2023 = df_2023[df_2023['Valeur fonciere'].notnull()]
df_2023.fillna(0, inplace=True)

#df.head()
df_2023.info()
#print(df.isnull().sum())
#print(df.shape)  


<class 'pandas.core.frame.DataFrame'>
Index: 3740127 entries, 0 to 3799406
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date mutation              object 
 1   Nature mutation            object 
 2   Valeur fonciere            object 
 3   Voie                       object 
 4   Code postal                float64
 5   Commune                    object 
 6   Code departement           object 
 7   Code commune               int64  
 8   Nombre de lots             int64  
 9   Code type local            float64
 10  Type local                 object 
 11  Surface reelle bati        float64
 12  Nombre pieces principales  float64
 13  Surface terrain            float64
dtypes: float64(5), int64(2), object(7)
memory usage: 428.0+ MB


Nous allons également modifier les types des variables pour mieux les manipuler lors des analyses :

- Date mutation devra être de type datetime 
- Nature mutation est préférablement un type catégorie pour optimiser la mémoire
- Valeur fonciere devra être de type float car représentant un montant financier
- Voie est un type category 
- Code postal sera un category
- Commune est un type category
- Code departement est de type category 
- Code commune est de type category.

In [6]:
df_2023['Date mutation']=pd.to_datetime(df_2023['Date mutation'], errors='coerce')
df_2023['Valeur fonciere']=df_2023['Valeur fonciere'].replace({'\xa0':'',',':'.'}, regex=True)
df_2023['Valeur fonciere']=pd.to_numeric(df_2023['Valeur fonciere'], errors='coerce')

categorical_columns = ['Voie', 'Nature mutation', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Code type local', 'Type local']
df_2023[categorical_columns]=df_2023[categorical_columns].apply(lambda x: x.astype('category'))

df_2023.info()
df_2023.head()


<class 'pandas.core.frame.DataFrame'>
Index: 3740127 entries, 0 to 3799406
Data columns (total 14 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   Date mutation              datetime64[ns]
 1   Nature mutation            category      
 2   Valeur fonciere            float64       
 3   Voie                       category      
 4   Code postal                category      
 5   Commune                    category      
 6   Code departement           category      
 7   Code commune               category      
 8   Nombre de lots             int64         
 9   Code type local            category      
 10  Type local                 category      
 11  Surface reelle bati        float64       
 12  Nombre pieces principales  float64       
 13  Surface terrain            float64       
dtypes: category(8), datetime64[ns](1), float64(4), int64(1)
memory usage: 270.7 MB


,Date mutation,Nature mutation,Valeur fonciere,Voie,Code postal,Commune,Code departement,Code commune,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
0,2023-05-01,Vente,1070000.0,DES HETRES,1630.0,ST-GENIS-POUILLY,01,354,1,3.0,Dépendance,0.0,0.0,0.0
1,2023-05-01,Vente,1070000.0,DES HETRES,1630.0,ST-GENIS-POUILLY,01,354,1,2.0,Appartement,233.0,8.0,0.0
2,2023-05-01,Vente,1070000.0,DES HETRES,1630.0,ST-GENIS-POUILLY,01,354,1,3.0,Dépendance,0.0,0.0,0.0
3,2023-03-01,Vente,152200.0,DE PONCIN,1450.0,SERRIERES-SUR-AIN,01,404,0,1.0,Maison,64.0,3.0,988.0
4,2023-05-01,Vente,269000.0,DE L'AUBEPIN,1800.0,SAINT-JEAN-DE-NIOST,01,361,0,1.0,Maison,73.0,3.0,835.0


In [7]:
#### SAVE DATAFRAME ####

df_2023.to_csv('df_2023.csv', index=False)